# Thai Text Generation from Few-shot learning with Flax GPT-2-base

I use quote from Albert Einstein at https://th.wikiquote.org/wiki/อัลเบิร์ต_ไอน์สไตน์/. It's CC-BY-SA 3.0 License.

In [1]:
!pip install padthai

     |████████████████████████████████| 2.6 MB 12.5 MB/s 
     |████████████████████████████████| 636 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
data = [
        "กลศาสตร์ควอนตัมนั้นน่าอัศจรรย์ แต่เสียงในหัวใจของฉันบอกว่ามันยังไม่ใช่ความจริง ทฤษฎีบอกอะไรได้มากมาย แต่มันยังไม่พาเราเข้าใกล้ความลับของพระเจ้า ฉันคิดว่า พระเจ้าไม่ได้ทอยลูกเต๋าหรอก",
        "1 ชั่วโมงที่ชายหนุ่มนั่งเคียงหญิงสาวผ่านไปไวราวกับ 1 นาที แต่ 1 นาทีที่เขานั่งบนเตาที่ร้อน ผ่านไปราวกับ 1 ชั่วโมง นี่คือสัมพัทธภาพ",
        "เชาว์ปัญญาทำให้เรามีความกระจ่างชัดในเรื่อง ความสัมพันธ์ระหว่างกันของวิธีการกับเป้าหมาย แต่ลำพังการคิดหาเหตุผลนั้นมิอาจทำให้เรามีความรู้สึกเกี่ยวกับเป้าหมายพื้นฐานและเป้าหมายสุดท้ายได้ การทำความกระจ่างชัดเกี่ยวกับเป้าหมายพื้นฐานเหล่านี้ และการประเมินค่าก็ดี, ทั้งการทำให้เป้าหมายพื้นฐานเหล่านี้แน่นเหนียวในชีวิตทางอารมย์ความรู้สึกของปัจเจกบุคคลก็ดี สำหรับข้าพเจ้าแล้วดูเหมือนเป็นที่แน่นอนว่าหน้าที่สำคัญที่สุด ซึ่งศาสนาต้องแสดงบทบาทในชีวิตทางสังคมของมนุษย์",
        "สมการมีความสำคัญต่อข้าพเจ้ามากกว่า, เพราะการเมืองเป็นเรื่องสำหรับปัจจุบันกาล, แต่สมการเป็นบางสิ่งบางอย่างสำหรับนิรันดร์กาล",
        "มีไม่กี่คนหรอกที่เห็นด้วยตา และรู้สึกด้วยหัวใจของตนเอง",
        "พระเจ้ามิได้ทรงเล่นลูกเต๋ากับเอกภพเป็นแน่",
        "จินตนาการสำคัญกว่าความรู้",
        "การเมืองนั้นแสนสั้น แต่สมการคงอยู่ชั่วนิรันดร์",
        "หากข้อเท็จจริงไม่เหมาะกับทฤษฎี, จงเปลี่ยนข้อเท็จจริงนั้นเสีย",
        "อย่าเป็นคนที่มุ่งหวังเพียงความสำเร็จ แต่จงมุ่งหวังความเป็นมนุษย์ที่มีคุณค่า",
        "ความกลัวความตายเป็นความกลัวที่ไม่มีเหตุผลที่สุดของมนุษย์",
        "มีเพียงชีวิตเพื่อผู้อื่นเท่านั้น ที่มีคุณค่าแก่การมีชีวิต",
        "ไม่สามารถรักษาสันติได้ด้วยกำลัง หากแต่ด้วยความเข้าใจ",
        "ความประสบความสำเร็จในชีวิตเท่ากับ A ดังนั้น A = x + y + z เมื่อ x เท่ากับงาน y เท่ากับเที่ยวเล่น และ z เท่ากับ หุบปากสนิท",
        "ข้าพเจ้าไม่อาจทราบได้ว่าสงครามโลกครั้งที่ 3 ใช้อะไรสู้กัน แต่สงครามโลกครั้งที่ 4 จะต่อสู้กันด้วยท่อนไม้และก้อนหิน",
        "สิ่งสำคัญอย่างยิ่งก็คือ อย่าได้หยุดตั้งคำถาม ความกระหายใฝ่เรียนรู้นั้นมีเหตุผลในตัวมันเองที่จะต้องดำรงอยู่ ช่วยไม่ได้เลยที่เราจะรู้สึกครั่นคร้ามเมื่อได้ครุ่นคิดถึงความลึกลับแห่งนิจนิรันดร์ชีวิต และลักษณะอันน่าพิศวงของสัจจะ แค่เพียงได้คิดคำนึงถึงความลึกลับเหล่านี้วันละนิดก็เพียงพอแล้ว จงอย่าได้สูญเสียความกระหายใฝ่รู้อันศักดิ์สิทธิ์นี้"
]

In [3]:
data[:9]

['กลศาสตร์ควอนตัมนั้นน่าอัศจรรย์ แต่เสียงในหัวใจของฉันบอกว่ามันยังไม่ใช่ความจริง ทฤษฎีบอกอะไรได้มากมาย แต่มันยังไม่พาเราเข้าใกล้ความลับของพระเจ้า ฉันคิดว่า พระเจ้าไม่ได้ทอยลูกเต๋าหรอก',
 '1 ชั่วโมงที่ชายหนุ่มนั่งเคียงหญิงสาวผ่านไปไวราวกับ 1 นาที แต่ 1 นาทีที่เขานั่งบนเตาที่ร้อน ผ่านไปราวกับ 1 ชั่วโมง นี่คือสัมพัทธภาพ',
 'เชาว์ปัญญาทำให้เรามีความกระจ่างชัดในเรื่อง ความสัมพันธ์ระหว่างกันของวิธีการกับเป้าหมาย แต่ลำพังการคิดหาเหตุผลนั้นมิอาจทำให้เรามีความรู้สึกเกี่ยวกับเป้าหมายพื้นฐานและเป้าหมายสุดท้ายได้ การทำความกระจ่างชัดเกี่ยวกับเป้าหมายพื้นฐานเหล่านี้ และการประเมินค่าก็ดี, ทั้งการทำให้เป้าหมายพื้นฐานเหล่านี้แน่นเหนียวในชีวิตทางอารมย์ความรู้สึกของปัจเจกบุคคลก็ดี สำหรับข้าพเจ้าแล้วดูเหมือนเป็นที่แน่นอนว่าหน้าที่สำคัญที่สุด ซึ่งศาสนาต้องแสดงบทบาทในชีวิตทางสังคมของมนุษย์',
 'สมการมีความสำคัญต่อข้าพเจ้ามากกว่า, เพราะการเมืองเป็นเรื่องสำหรับปัจจุบันกาล, แต่สมการเป็นบางสิ่งบางอย่างสำหรับนิรันดร์กาล',
 'มีไม่กี่คนหรอกที่เห็นด้วยตา และรู้สึกด้วยหัวใจของตนเอง',
 'พระเจ้ามิได้ทรงเล่นลูกเต๋ากับเอ

In [11]:
data[9:]

['อย่าเป็นคนที่มุ่งหวังเพียงความสำเร็จ แต่จงมุ่งหวังความเป็นมนุษย์ที่มีคุณค่า',
 'ความกลัวความตายเป็นความกลัวที่ไม่มีเหตุผลที่สุดของมนุษย์',
 'มีเพียงชีวิตเพื่อผู้อื่นเท่านั้น ที่มีคุณค่าแก่การมีชีวิต',
 'ไม่สามารถรักษาสันติได้ด้วยกำลัง หากแต่ด้วยความเข้าใจ',
 'ความประสบความสำเร็จในชีวิตเท่ากับ A ดังนั้น A = x + y + z เมื่อ x เท่ากับงาน y เท่ากับเที่ยวเล่น และ z เท่ากับ หุบปากสนิท',
 'ข้าพเจ้าไม่อาจทราบได้ว่าสงครามโลกครั้งที่ 3 ใช้อะไรสู้กัน แต่สงครามโลกครั้งที่ 4 จะต่อสู้กันด้วยท่อนไม้และก้อนหิน',
 'สิ่งสำคัญอย่างยิ่งก็คือ อย่าได้หยุดตั้งคำถาม ความกระหายใฝ่เรียนรู้นั้นมีเหตุผลในตัวมันเองที่จะต้องดำรงอยู่ ช่วยไม่ได้เลยที่เราจะรู้สึกครั่นคร้ามเมื่อได้ครุ่นคิดถึงความลึกลับแห่งนิจนิรันดร์ชีวิต และลักษณะอันน่าพิศวงของสัจจะ แค่เพียงได้คิดคำนึงถึงความลึกลับเหล่านี้วันละนิดก็เพียงพอแล้ว จงอย่าได้สูญเสียความกระหายใฝ่รู้อันศักดิ์สิทธิ์นี้']

In [4]:
from padthai import *

2022-10-07 09:06:21.622296: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


2022-10-07 09:06:22,247 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2022-10-07 09:06:23,938 - git.cmd - DEBUG - Popen(['git', 'version'], cwd=/workspace/pythainlp-work/padthai/notebooks, universal_newlines=False, shell=None, istream=None)
2022-10-07 09:06:23,956 - git.cmd - DEBUG - Popen(['git', 'version'], cwd=/workspace/pythainlp-work/padthai/notebooks, universal_newlines=False, shell=None, istream=None)


In [5]:
flaxgpt2_model = FlaxGPT2FewShot('./flex-model2')

2022-10-07 09:06:24,635 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2022-10-07 09:06:25,751 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /flax-community/gpt2-base-thai/resolve/main/vocab.json HTTP/1.1" 200 0
2022-10-07 09:06:25,987 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2022-10-07 09:06:27,045 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /flax-community/gpt2-base-thai/resolve/main/config.json HTTP/1.1" 200 0


In [6]:
flaxgpt2_model.train(
    train_data=data[:9],
    test_data=data[9:],
    logging_dir='./log_model_flex',
    num_train_epochs=1, # change!!!
    train_size=0.9,
    batch_size=8,
    save_every_epochs=False
)

2022-10-07 09:06:35,689 - root - DEBUG - finding max_length...
2022-10-07 09:06:35,741 - root - DEBUG - use train_data and test_data...


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


2022-10-07 09:06:36,236 - wandb.jupyter - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2022-10-07 09:06:37,899 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-07 09:06:38,215 - urllib3.connectionpool - DEBUG - https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 521
2022-10-07 09:06:38,221 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-07 09:06:38,535 - urllib3.connectionpool - DEBUG - https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 343


wandb: Currently logged in as: wannaphong. Use `wandb login --relogin` to force relogin


2022-10-07 09:06:38,572 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/workspace/pythainlp-work/padthai, universal_newlines=False, shell=None, istream=<valid stream>)


/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,10.088700,9.938056


***** Running Evaluation *****
  Num examples = 7
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 7
  Batch size = 64


Saving model checkpoint to ./flex-model2
Configuration saved in ./flex-model2/config.json
Model weights saved in ./flex-model2/pytorch_model.bin


In [7]:
flaxgpt2_model.gen("ความก้าวหน้าของมนุษย์",max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['ความก้าวหน้าของมนุษย์ทั้งปวง เพราะยุคสมัยปัจจุบันนี้ทุกคนมีชีวิตของตนเอง เรียกว่า “อินเดีย” (อังกฤษ : Indians) เป็นต้นกำเนิดของความไม่แน่นอน ซึ่งไม่สามารถตอบสนองได้ เป็นผลมาจากการเปลี่ยนแปลงครั้งสำคัญ ๆ ใน',
 'ความก้าวหน้าของมนุษย์และสิ่งแวดล้อม ซึ่งก็หมายความว่าความดีความชั่วมีความหมายสูงและจะทำให้คนมีความสุขกับการเรียนการสอนและมีความสุขกับโครงการดีๆมากมาย นั่นคือข้อความส่วนตัวที่ทุกคนต้องเคยส่งมาเพื่อสื่อความหมายที่ถ',
 'ความก้าวหน้าของมนุษย์ แต่มันมีเหตุผลนะครับที่ทำให้มนุษย์ยังคงเป็นสิ่งต้องห้ามและไม่มีหลักการรับรองว่ามันจะเป็นสิ่งที่ถูกต้องหรือไม่ถ้าหากนักวิทยาศาสตร์รับรองว่า “มันจะเป็นสิ่งที่',
 'ความก้าวหน้าของมนุษย์ เป็นผู้ใหญ่ที่มีบุคลิกที่ซื่อสัตย์ และมีทักษะในการแก้ไขปัญหาต่าง ๆ ส่งผลให้เป็นผู้รักความสงบและมีมารยาทในการตัดสินใจ เป็นแบบอย่างในการดำเนินชีวิต ด้วยคำพูด ร้อยกรองเรื่องความเป',
 'ความก้าวหน้าของมนุษย์ และยังทำให้มนุษย์สามารถควบคุมได้ด้วยตนเอง\nนอกจากจะเป็นอาชีพที่อยู่ในสายเลือดแล้ว หนุ่มสาวยุคใหม่เริ่มต้นสร้างครอบครัวกันเอง เพราะครอบครัวที่มีลูกสาวน้อยต้องทำตัวเ

In [8]:
flaxgpt2_model.gen("เทคโนโลยี",max_length=160)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['เทคโนโลยีการสื่อสารและภาพลักษณ์ของประเทศไทยด้วยอุปกรณ์ทางเทคนิค: การวิจัยทางเทคนิค การวิจัยทางเทคนิคการสื่อสารและความบันเทิง - บทความ - ธรรมะในชีวิตประจำวัน สำหรับคนในสังคม บทความ - ธรรมะในชีวิตประจำวัน สำหรับคนในสังคม\nวิจัยในระดับมหาวิทยาลัยได้เก็บรวบรวมข้อมูลเกี่ยวกับการใช้อุปกรณ์ทางเทคนิคที่เหมาะสมและเหมาะสม สำหรับการสื่อสารในปัจจุบัน\nวิจัยในระดั',
 'เทคโนโลยีดิจิตอล” ที่ต้องเปิดให้เข้าถึงได้นี้คือ การนำเสนอเนื้อหาผ่านการวิเคราะห์ด้วยนวัตกรรมสมัยใหม่\nสิ่งที่ได้จากการนำเสนอเนื้อหานั้นคือ เราจะสามารถนำเสนอเนื้อหาอย่างมีประสิทธิภาพมากขึ้น โดยทุกองค์กรจะมีการตัดสินใจเชิงนโยบายอย่างเป็นธรรม ทั้งด้านนโยบาย โครงสร้างพื้นฐาน ระบบการศึกษา รวมถึงเครือข่ายดิจิตอลที่สำคัญอย่างสม่ำเสมอ เพื่อนำไปสู่การพั',
 'เทคโนโลยีสารสนเทศและการสื่อสาร ได้นำวิทยาการและเทคโนโลยีสารสนเทศมาใช้ในการปฏิบัติงาน เป็นการพัฒนาและเพิ่มประสิทธิภาพการทำงานของหน่วยงาน รวมทั้งการขับเคลื่อนงาน ขององค์กรมีข้อบังคับและกฎหมายกำหนดไว้ ดังนี้\n1.1.4\tโครงการการพัฒนาศักยภาพด้านความรู้ความสามารถเชิงวิชาการทางเทคโนโลยีสารสนเทศแ

In [9]:
flaxgpt2_model.gen("ความรัก",max_length=160)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['ความรัก, ช่วยให้ความจำ, ทุกชนิดของวัสดุที่มีความซับซ้อนในการกันน้ำมันป้องกันหิน\nเครื่องมือแบบพกพาร่วมสมัยกับหน่วยย่อยของการออกแบบใหม่ในปี 1892 ในชื่อ ‘Alexander: The New Car’ นักปรัชญา, สังคมศาสตร์, และบรรดาผู้เขียนของความคิดนี้พิมพ์เป็นที่นิยมมากในพื้นที่ภูเขาและภูเขาในแคลิฟอร์เนีย\nกีตาร์มันเป็นรูปแบบที่ถูกต้',
 'ความรักและอิสระได้ที่นี่ เราพักที่นั่นหนึ่งคืนในเดือนมิถุนายน และมีความสุขมากที่ได้พัก 3 คืนบนรถตู้ที่เก่าแก่ของประวัติศาสตร์ และราคาที่ดี มีร้านกาแฟเล็ก ๆ และร้านอาหารที่น่ารัก เดอะที่ยอดเยี่ยมที่สุด ห้องพักมีขนาดกว้างขวาง และสะอาดมาก และสะดวกสบาย พนักงานเอาใจใส่ และมีความรู้ แน่นอนเราจะแนะนำให้มาเที่ยวจุดอื่นเพิ่มเติม\nฉันไม่เคยผิดหว',
 'ความรัก ที่จะมัดใจอีกฝ่ายให้เขารักกันทุกวัน\nเมื่อความรักครั้งนี้ถาโถมเข้ามามากจนแทบส่งผลร้ายไปถึงคนรัก มีปัญหาขัดแย้งกันบ้างเป็นบางอย่าง หรือ มีปัญหาอื่นๆ ตามมาให้พูดกันไปมา เพราะทุกคนรู้อยู่แก่ใจถึงความรัก ว่าเขาเป็นอย่างไร และถ้าพบก็จะสามารถจูบ และรักเขาไปด้วยกัน ดีขึ้นทั้งสองคน\nนั่นเป็นท',
 'ความรัก” (Sweet Love) และนี่คือ “ว่าที่น

In [10]:
flaxgpt2_model.gen("กาลอวกาศ",max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['กาลอวกาศสุดสะพรึงนี้\n“เมื่อเครื่องบินจำนวน 10 เครื่องตกลงจากบนยอดดอยมากขึ้นแล้ว ก็จะทำให้มัน “ดูคล้ายงูเห่า”” ซึ่งถือเป็นอันตรายมากสำหรับชาวพื้นเมืองในลาวตอนใต้ และทำให้พวกเขาต้องกลายเป็นคนไร้บ้',
 'กาลอวกาศก็มีอิทธิพลมากพอสมควรในการทำให้เกิดความต้องการและเป้าหมายทางการแพทย์เช่น การทำให้คนไข้สามารถรับยาได้เพื่อการดูดซึมที่ดีขึ้น การรักษาโรคเอดส์จึงเป็นมาตรการที่สามารถทำได้ง่ายและรวดเร็ว\nแต่ในทางปฏิบัติ',
 'กาลอวกาศได้สำเร็จ\nนอกจากนี้มันยังได้รับรางวัลจากแคมเปญใหม่ที่จัดทำขึ้นด้วยวัสดุที่ดียิ่งขึ้นและได้รับการคัดเลือกให้เป็นหนึ่งในนักสร้างอนุภาคที่ดีที่สุดในโลก โดยมีรายละเอียดดังนี้\nทีมงานได้ถ',
 'กาลอวกาศ ที่สร้างขึ้นในช่วงปลายศตวรรษที่ 19 เป็นต้นมา โดยใช้งบประมาณในการก่อสร้างกว่า 1,500 ล้านบาท\nโดยเครื่องบินที่ใช้สำหรับการเดินทางของเราได้ผ่านการพัฒนาและปรับปรุงหลายอย่าง เช่น มีการปรับปรุงรายละเอียดต่างๆ อีกมาก ทั้งโครงสร',
 'กาลอวกาศ และใช้เวลาเพียงครึ่งชั่วโมงในการถ่ายภาพของเขา โดยเขาได้ลั่นวาจาเอาไว้ว่า เขาจะไปดูหนัง และฟังเพลงในโรงภาพยนตร์ เพื่อหาประสบการณ์ใหม่ๆในการเรียนรู้ส